Codigo do tcc

In [1]:
!pip3 install transformers
!pip3 install dl-translate
!pip3 install emoji
!pip3 install tensorflow
!pip3 install numpy
!pip3 install tensorflow_hub

In [30]:
import tensorflow as tf
import numpy as np
import pandas as pd
import dl_translate as dlt
from sklearn.metrics import precision_score, recall_score, f1_score

In [3]:
mt = dlt.TranslationModel()

In [4]:
def translater_frases(frase):
  text_hi = frase
  translater = mt.translate(text_hi, source=dlt.lang.PORTUGUESE, target=dlt.lang.ENGLISH)
  return translater

def classification_frasesR(frase):
  input = tokenizerR.encode(frase, return_tensors="tf")
  output = modelR.predict(input)[0]
  prediction = tf.nn.softmax(output, axis=1).numpy()
  sentiment = np.argmax(prediction) 

  return sentiment

def classification_frasesM(frase):
  inputs = tokenizerM(frase, return_tensors="tf")
  outputs = modelM(inputs)
  logits = outputs.logits
  sentiment = tf.argmax(logits, axis=1).numpy()[0]

  return sentiment

def classification_frasesC(frase):
  sentiment = sentiment_task(frase)

  sentiment = sentiment[0]['label']

  sentiment = sentiment.replace('positive', '1').replace('negative', '-1').replace('neutral', '0')

  return sentiment


## Modelo de classificação de Tweets, treinado com 23 milhões de exemplos em inglês, sobre covid.

In [5]:
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification
  
tokenizerR = AutoTokenizer.from_pretrained("rabindralamsal/BERTsent")

modelR = TFAutoModelForSequenceClassification.from_pretrained("rabindralamsal/BERTsent")


2023-09-05 09:37:00.029032: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-09-05 09:37:00.029338: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

All the layers of TFRobertaForSequenceClassification were initialized from the model checkpoint at rabindralamsal/BERTsent.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions 

In [6]:
example_tweet = "The NEET exams show our Govt in a poor light: unresponsiveness to genuine concerns; admit cards not delivered to aspirants in time; failure to provide centres in towns they reside, thus requiring unnecessary & risky travels. What a disgrace to treat our #Covid warriors like this!"
    
input = tokenizerR.encode(example_tweet, return_tensors="tf")
output = modelR.predict(input)[0]
prediction = tf.nn.softmax(output, axis=1).numpy()

sentiment = np.argmax(prediction)
    
print(prediction)
print(sentiment)

1/1 [==============================] - 5s 5s/step
[[0.97267216 0.02368475 0.00364307]]
0


## Este é um modelo multilíngue baseado em XLM-roBERTa treinado em aproximadamente 198 milhões de tweets e ajustado para análise de sentimento. O ajuste fino do sentimento foi feito em 8 idiomas (Ar, En, Fr, De, Hi, It, Sp, Pt).

In [7]:
from transformers import pipeline
model_path = "cardiffnlp/twitter-xlm-roberta-base-sentiment"
sentiment_task = pipeline("sentiment-analysis", model=model_path, tokenizer=model_path)
sentiment_task("que dia pessimo")


[{'label': 'negative', 'score': 0.9488404989242554}]

In [9]:
dataset = pd.read_csv("./TweetSentBR.csv")

In [10]:
dataset.drop(columns = ['id', 'id_twitter'],inplace=True)

In [11]:
dataset

,text,sentiment
0,Que coisa linda! O Programa #encontro estava m...,1
1,"Por mais #Encontro com as Irmãs Galvão, adorei...",1
2,Mr. CATRA @OficialMrCatra lançando sua nova mú...,1
3,quem viu aquela lutadora modela barbuda tatuad...,0
4,Tô passada com esse cara.... quanta merda pode...,-1
...,...,...
11560,eu ja to aqui pronto pro #MasterChefBR mas ain...,-1
11561,MALUCO! Uma coisa que eu não tenho coragem é e...,-1
11562,#MaisVoce @ANAMARIABRAGA está linda @RedeGlobo,1
11563,"Que orgulho de ti, @sportrecife! #Encontro",1


In [10]:
positive = dataset[dataset['sentiment'] == '1'].head(10)

In [11]:
positive['textEnglish'] = positive['text'].apply(translater_frases)

In [12]:
positive['sentimentEnglish'] = positive['textEnglish'].apply(classification_frasesR)

1/1 [==============================] - 0s 58ms/step


In [13]:
positive['sentimentEnglish'] = positive['sentimentEnglish'].replace(2, '1').replace(0, '-1').replace(1, '0')

In [14]:
nomepositivo= 'positive.xlsx'
positive.to_excel(nomepositivo, index=False)

In [15]:
negative = dataset[dataset['sentiment'] == '-1'].head(10)

In [16]:
negative['textEnglish'] = negative['text'].apply(translater_frases)

In [17]:
negative['sentimentEnglish'] = negative['textEnglish'].apply(classification_frasesR)

1/1 [==============================] - 0s 111ms/step


In [18]:
negative['sentimentEnglish'] = negative['sentimentEnglish'].replace(2, '1').replace(0, '-1').replace(1, '0')

In [19]:

negative.to_excel('negative.xlsx', index=False)

In [12]:
dataset = pd.read_csv("./datasetcompolaridade.csv")

In [13]:
dataset.drop(columns = ['Unnamed: 0', 'tweets_translater_unicamp'],inplace=True)

In [14]:
positive = dataset[dataset['polaridade'] == 1].head(100)

In [15]:
negative = dataset[dataset['polaridade'] == -1].head(100)

In [16]:
neutral = dataset[dataset['polaridade'] == 0].head(100)

In [17]:
positive['polaridadeClassificationPortugues'] = positive['tweets_original'].apply(classification_frasesC)

In [18]:
positive['polaridadeClassificationIngles'] = positive['tweets_translater_face'].apply(classification_frasesC)

In [19]:
positive['polaridadeClassificationPortugues'] = positive['polaridadeClassificationPortugues'].replace(2, 1).replace(0, -1).replace(1, 0)

In [20]:
positive['polaridadeClassificationIngles'] = positive['polaridadeClassificationIngles'].replace(2, 1).replace(0, -1).replace(1, 0)

In [21]:
positive

,tweets_original,tweets_translater_face,polaridade,polaridadeClassificationPortugues,polaridadeClassificationIngles
0,Que coisa linda! O Programa #encontro estava m...,What a beautiful thing! The #touch program was...,1.0,1,1
1,"Por mais #Encontro com as Irmãs Galvão, adorei...","For more I met with the Galvan Sisters, I love...",1.0,1,1
2,Mr. CATRA lançando sua nova música PPK CHORA ...,Mr. CATRA releases his new song PPK CHORA on k...,1.0,0,0
5,"Cibele arrasou, humilhou!! #VideoShowAoVivo","The cave has broken, humiliated!! #VideoShowAo...",1.0,1,-1
9,A Cearamirinense mais famosa do Brasil no #Ede...,The most famous Cearamirinense in Brazil at #E...,1.0,1,1
...,...,...,...,...,...
209,mal começou o programa e eu já tô chorando #Al...,It’s hard to start the program and I’m already...,1.0,-1,-1
210,Braulio sempre com suas poesias 😍 #Encontro,Braulio always with his poems,1.0,1,1
212,#ConversaComBial\r\nAnsiosa pra assistir a con...,#ConversaComBial Anxiosa to watch today's talk...,1.0,1,1
216,Rafael ainda nem nasceu e já escuta #anjo #40...,"I'm not sure I'm going to do it, I'm not sure ...",1.0,0,0


In [109]:
positive['polaridadeClassificationPortugues'] = positive['polaridadeClassificationPortugues'].astype(float)
precisionPositivePortugues = precision_score(positive['polaridade'], positive['polaridadeClassificationPortugues'], average='macro')
recallPositivePortugues = recall_score(positive['polaridade'], positive['polaridadeClassificationPortugues'], average='macro')
f1PositivePortugues = f1_score(positive['polaridade'], positive['polaridadeClassificationPortugues'], average='macro')

/home/lincoln81/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [110]:
f1PositivePortugues

0.2900188323917137

In [96]:
positive['polaridadeClassificationIngles'] = positive['polaridadeClassificationIngles'].astype(float)
precisionPositiveIngles = precision_score(positive['polaridade'], positive['polaridadeClassificationIngles'], average='macro')
recallPositiveIngles = recall_score(positive['polaridade'], positive['polaridadeClassificationIngles'], average='macro')
f1PositiveIngles = f1_score(positive['polaridade'], positive['polaridadeClassificationIngles'], average='macro')

/home/lincoln81/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [97]:
f1PositiveIngles

0.26016260162601623

In [22]:
negative['polaridadeClassificationPortugues'] = negative['tweets_original'].apply(classification_frasesC)
negative['polaridadeClassificationIngles'] = negative['tweets_translater_face'].apply(classification_frasesC)

In [23]:
negative['polaridadeClassificationPortugues'] = negative['polaridadeClassificationPortugues'].replace(2, 1).replace(0, -1).replace(1, 0)
negative['polaridadeClassificationIngles'] = negative['polaridadeClassificationIngles'].replace(2, 1).replace(0, -1).replace(1, 0)

In [24]:
negative

,tweets_original,tweets_translater_face,polaridade,polaridadeClassificationPortugues,polaridadeClassificationIngles
4,Tô passada com esse cara.... quanta merda pode...,How much shit can come out of someone’s mouth ...,-1.0,-1,-1
8,Esse japa não entendi porra nenhuma de orquíde...,This japa I didn't understand any of orchids. ...,-1.0,-1,-1
19,Aí vc fica até 01h00 assistindo #MasterChefBR ...,Here you stay until 1h00 watching #MasterChefB...,-1.0,-1,-1
21,Imagina que insuportável ter de dar de comer p...,Imagine how unbearable it is to have to give f...,-1.0,-1,-1
25,o prato do Vitor B ficou ruim MEU DEUS TO PASS...,The Vitor B's dish was bad my God to pass bad ...,-1.0,-1,-1
...,...,...,...,...,...
298,ahhhh... uma semana pra ter #MasterChefBR de n...,Ahhhh... a week to have #MasterChefBR again... :(,-1.0,-1,-1
304,A Paola kkkkkkkkkk\r\n#MasterChefBR,The Paola kkkkkkkk #MasterChefBR,-1.0,0,0
305,Não tá dando pra assistir MasterChef com esse ...,You don’t want to see MasterChef with that tim...,-1.0,-1,-1
309,"O crush da vida saiu, estou bem triste #Master...","The crush of life has gone out, I'm very sad #...",-1.0,-1,-1


In [86]:
negative['polaridadeClassificationPortugues'] = negative['polaridadeClassificationPortugues'].astype(float)
precisionNegativePortugues = precision_score(negative['polaridade'], negative['polaridadeClassificationPortugues'], average='micro')
recallNegativePortugues = recall_score(negative['polaridade'], negative['polaridadeClassificationPortugues'], average='micro')
f1NegativePortugues = f1_score(negative['polaridade'], negative['polaridadeClassificationPortugues'], average='micro')


In [87]:
f1NegativePortugues

0.76

In [88]:
negative['polaridadeClassificationIngles'] = negative['polaridadeClassificationIngles'].astype(float)
precisionNegativeIngles = precision_score(negative['polaridade'], negative['polaridadeClassificationIngles'], average='micro')
recallNegativeIngles = recall_score(negative['polaridade'], negative['polaridadeClassificationIngles'], average='micro')
f1NegativeIngles = f1_score(negative['polaridade'], negative['polaridadeClassificationIngles'], average='micro')

In [89]:
f1NegativeIngles

0.62

In [25]:
neutral['polaridadeClassificationPortugues'] = neutral['tweets_original'].apply(classification_frasesC)

In [26]:
neutral.dropna(inplace=True)

In [27]:
neutral['polaridadeClassificationIngles'] = neutral['tweets_translater_face'].apply(classification_frasesC)

In [28]:
neutral['polaridadeClassificationPortugues'] = neutral['polaridadeClassificationPortugues'].replace(2, 1).replace(0, -1).replace(1, 0)
neutral['polaridadeClassificationIngles'] = neutral['polaridadeClassificationIngles'].replace(2, 1).replace(0, -1).replace(1, 0)

In [29]:
neutral

,tweets_original,tweets_translater_face,polaridade,polaridadeClassificationPortugues,polaridadeClassificationIngles
3,quem viu aquela lutadora modela barbuda tatuad...,Who saw that fighter model tattooed? #MasterCh...,0.0,-1,0
6,Em Minas sacolé é chup chup! #videoshowaovivo,In Minas sacolé is chup chup! #videohowaovivo,0.0,0,0
7,Meu prato MasterChef de sardinha enlatada seri...,My MasterChef dish of enlated sardine would be...,0.0,0,0
13,Fatima Bernardes sobre o sotaque hahaha coerên...,Fatima Bernardes on the accent hahaha professi...,0.0,1,1
23,Olá gostaria da participação do Fiuk em algum...,"Hello, I would like Fiuk to participate in som...",0.0,0,0
...,...,...,...,...,...
669,#MasterChefBR espero que na próxima semana a M...,#MasterChefBR I hope that next week Miriam wil...,0.0,-1,-1
670,#Encontro Cuidado ao ingerir broto de samambai...,Be careful when eating samambae outbreak becau...,0.0,-1,-1
671,Lembrei q hj tem #MasterChefBR e já me deu fome🙄,I remember q hj has #MasterChefBR and it’s alr...,0.0,-1,0
676,Sardinha é muito boa em cima da pizza caseira ...,Sardinia is very good over the home pizza #Mas...,0.0,1,1


In [90]:
neutral['polaridadeClassificationPortugues'] = neutral['polaridadeClassificationPortugues'].astype(float)
precisionNeutralPortugues = precision_score(neutral['polaridade'], neutral['polaridadeClassificationPortugues'], average='micro')
recallNeutralPortugues = recall_score(neutral['polaridade'], neutral['polaridadeClassificationPortugues'], average='micro')
f1NeutralPortugues = f1_score(neutral['polaridade'], neutral['polaridadeClassificationPortugues'], average='micro')

In [91]:
f1NeutralPortugues

0.696969696969697

In [92]:
neutral['polaridadeClassificationIngles'] = neutral['polaridadeClassificationIngles'].astype(float)
precisionNeutralIngles = precision_score(neutral['polaridade'], neutral['polaridadeClassificationIngles'], average='micro')
recallNeutralIngles = recall_score(neutral['polaridade'], neutral['polaridadeClassificationIngles'], average='micro')
f1NeutralIngles = f1_score(neutral['polaridade'], neutral['polaridadeClassificationIngles'], average='micro')


In [93]:
f1NegativeIngles

0.62

In [111]:
b2w = pd.read_csv("./b2w.csv")

In [115]:
b2w.drop(columns = ['original_index', 'review_text_processed', 'review_text_tokenized', 'rating', 'kfold_polarity', 'kfold_rating'], inplace=True)

In [117]:
b2w.dropna(inplace=True)

In [119]:
b2wPositive = b2w[b2w['polarity'] == 1].head(100)

In [120]:
b2wPositive

,review_text,polarity
0,Bem macio e felpudo...recomendo. Preço imbatí...,1.0
1,Produto excepcional! recomendo!!! inovador e ...,1.0
3,Bom custo beneficio. Adequado para pessoas que...,1.0
5,"ótmo produto atendeu as expectativas, a entreg...",1.0
6,"GOSTEI MUITO DO PRODUTO! TEM ÓTIMA QUALIDADE, ...",1.0
...,...,...
152,O produto é bem acabada mais a imagem n é oque...,1.0
153,Produto de boa qualidade e que atendeu as .min...,1.0
156,Roteador ótimo com diversos recursos. Chegou n...,1.0
157,"Satisfeita com a compra, exatamente como desc...",1.0


In [123]:
b2wPositive['review_translater_face'] = b2wPositive['review_text'].apply(translater_frases)

In [129]:
b2wPositive['polaridadeClassificationPortugues'] = b2wPositive['review_text'].apply(classification_frasesC)

In [131]:
b2wPositive['polaridadeClassificationIngles'] = b2wPositive['review_translater_face'].apply(classification_frasesC)

,review_text,review_translater_face,polaridade,polaridadeClassificationPortugues,polaridadeClassificationIngles
0,Bem macio e felpudo...recomendo. Preço imbatí...,Good soft and smooth I recommend. unbeatable p...,1.0,1,1
1,Produto excepcional! recomendo!!! inovador e ...,Excellent product! recommend!!! innovative and...,1.0,1,1
3,Bom custo beneficio. Adequado para pessoas que...,Good cost benefit. Suitable for people who do ...,1.0,1,1
5,"ótmo produto atendeu as expectativas, a entreg...","The product fulfilled expectations, the delive...",1.0,1,1
6,"GOSTEI MUITO DO PRODUTO! TEM ÓTIMA QUALIDADE, ...","It has excellent quality, and makes it very ea...",1.0,1,1
...,...,...,...,...,...
152,O produto é bem acabada mais a imagem n é oque...,The product is well finished more the image n ...,1.0,0,1
153,Produto de boa qualidade e que atendeu as .min...,Good quality product that meets my expectations.,1.0,1,1
156,Roteador ótimo com diversos recursos. Chegou n...,It arrived in the box in perfect condition and...,1.0,1,1
157,"Satisfeita com a compra, exatamente como desc...","Satisfied with the purchase, exactly as descri...",1.0,1,1


In [134]:
b2wPositive['polaridadeClassificationPortugues'] = b2wPositive['polaridadeClassificationPortugues'].astype(float)
precisionB2wPositivePortugues = precision_score(b2wPositive['polaridade'], b2wPositive['polaridadeClassificationPortugues'], average='micro')
recallB2wPositivePortugues = recall_score(b2wPositive['polaridade'], b2wPositive['polaridadeClassificationPortugues'], average='micro')
f1B2wPositivePortugues = f1_score(b2wPositive['polaridade'], b2wPositive['polaridadeClassificationPortugues'], average='micro')


In [135]:
f1B2wPositivePortugues

0.85

In [153]:
b2wPositive['polaridadeClassificationIngles'] = b2wPositive['polaridadeClassificationIngles'].astype(float)
precisionB2wPositiveIngles = precision_score(b2wPositive['polaridade'], b2wPositive['polaridadeClassificationIngles'], average='micro')
recallB2wPositiveIngles = recall_score(b2wPositive['polaridade'], b2wPositive['polaridadeClassificationIngles'], average='micro')
f1B2wPositiveIngles = f1_score(b2wPositive['polaridade'], b2wPositive['polaridadeClassificationIngles'], average='micro')

In [154]:
f1B2wPositiveIngles

0.87

In [121]:
b2wNegative = b2w[b2w['polarity'] == 0].head(100)

In [136]:
b2wNegative['review_translater_face'] = b2wNegative['review_text'].apply(translater_frases)

In [140]:
b2wNegative['polaridadeClassificationPortugues'] = b2wNegative['review_text'].apply(classification_frasesC)

In [144]:
b2wNegative['polaridadeClassificationIngles'] = b2wNegative['review_translater_face'].apply(classification_frasesC)

In [150]:
b2wNegative.drop(columns='polarity', inplace=True)

In [151]:
b2wNegative

,review_text,review_translater_face,polaridade,polaridadeClassificationPortugues,polaridadeClassificationIngles
2,recebi o produto antes do prazo mas veio com d...,I got the product before the deadline but came...,-1.0,-1,-1
10,"Não compra este produto, pois o mesmo veio com...",Don't buy this product because the same came w...,-1.0,-1,-1
12,"Tive azar, pois o produto não ligou. Tive que ...",I was unfortunate because the product didn't c...,-1.0,-1,-1
13,Uma enganação! Esperei quase um mês e cancelar...,I waited almost a month and they canceled the ...,-1.0,-1,-1
25,Cancelei 10 minutos depois que efetuei a compr...,I cancelled 10 minutes after I made the purcha...,-1.0,-1,0
...,...,...,...,...,...
339,A entrega foi rápida o produto veio lacrado e...,Delivery was fast the product came tears and i...,-1.0,0,1
340,Comprei o carrinha em 8 dias foi entregue... n...,I bought the car in 8 days was delivered... in...,-1.0,0,0
344,ENGANAÇÃO ! NÃO FUNCIONA NO BRASIL ! Esperei q...,I waited almost a month and I received a phone...,-1.0,-1,-1
346,"A capa tem uma apresentação legal, mas eu comp...","The cover has a legal presentation, but I boug...",-1.0,-1,0


In [156]:
b2wNegative['polaridadeClassificationPortugues'] = b2wNegative['polaridadeClassificationPortugues'].astype(float)
precisionB2wNegativePortugues = precision_score(b2wNegative['polaridade'], b2wNegative['polaridadeClassificationPortugues'], average='micro')
recallB2wNegativePortugues = recall_score(b2wNegative['polaridade'], b2wNegative['polaridadeClassificationPortugues'], average='micro')
f1B2wNegativePortugues = f1_score(b2wNegative['polaridade'], b2wNegative['polaridadeClassificationPortugues'], average='micro')


In [157]:
f1B2wNegativePortugues

0.81

In [158]:
b2wNegative['polaridadeClassificationIngles'] = b2wNegative['polaridadeClassificationIngles'].astype(float)
precisionB2wNegativeIngles = precision_score(b2wNegative['polaridade'], b2wNegative['polaridadeClassificationIngles'], average='micro')
recallB2wNegativeIngles = recall_score(b2wNegative['polaridade'], b2wNegative['polaridadeClassificationIngles'], average='micro')
f1B2wNegativeIngles = f1_score(b2wNegative['polaridade'], b2wNegative['polaridadeClassificationIngles'], average='micro')


In [159]:
f1B2wNegativeIngles

0.74